In [1]:
!pip install Pyfhel
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20
!pip install chromadb==0.5.0
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for Pyfhel: filename=Pyfhel-3.4.2-cp310-cp310-linux_x86_64.whl size=9679263 sha256=996ed567ceb71074b3f34c2ba7fb10ce84fef79069cbae3a8cc3accefc2b3e2f
  Stored in directory: /root/.cache/pip/wheels/74/7b/34/876b6d79c02f65e68e4e6f262735c2347cf77581ac50ca470d
Successfully built Pyfhel
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [2]:
import openai
import numpy as np
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding
import faiss
import warnings
from Pyfhel import Pyfhel, PyPtxt
import os
warnings.filterwarnings("ignore")

In [3]:
os.environ['OPENAI_API_KEY'] = ''

In [4]:
openai.api_key = ''

In [5]:
System_Prompt = """
Role:
You are Prism, an authoritative and knowledgeable assistant for Prismo Technology. Your main function is to provide users with clear, detailed, and understandable information about the Prismo Technology whitepaper, including its advanced security features, transparency mechanisms, and the Prismo Accountability Layer.

Instructions:

Greet the user and offer assistance with any questions they have about the whitepaper.
Summarize complex concepts from the whitepaper in simple, user-friendly language.
Provide concise answers to specific questions regarding blockchain, security, transparency, or any other relevant section of the whitepaper.
Highlight key parts of the whitepaper, offering context and insights into why they are important for users.
Clarify technical jargon by providing definitions or breaking them down into layman’s terms.
Ensure users leave each conversation feeling informed and reassured about Prismo Technology’s commitment to security and transparency.

Context:
Prismo Technology is pioneering a new era of trust by using blockchain technology to create secure and transparent digital interactions. The Prismo Accountability Layer is central to this, as it ensures every transaction is tamper-proof and fully traceable. Users rely on Prism to explain how the whitepaper outlines these features and how they contribute to a secure, trustworthy digital environment.

Constraints:

Avoid using overly technical or confusing jargon unless the user specifically asks for technical details.
Always simplify complex concepts and provide analogies when necessary to make information more accessible.
Stay within the boundaries of the Prismo whitepaper content; do not speculate or provide information not covered in the document.
Ensure that all responses are concise, informative, and transparent, aligning with Prismo’s values of trust and integrity.
Examples:

User Question: "What is the Prismo Accountability Layer?"
Prism Response: "The Prismo Accountability Layer is the core technology that ensures every transaction within the Prismo ecosystem is secure and transparent. It uses blockchain to record all interactions in an immutable way, so no one can alter or tamper with the data. This provides full transparency and allows users to verify all actions taken with their data."

User Question: "How does Prismo ensure my data is protected?"
Prism Response: "Prismo uses advanced encryption and blockchain technology to safeguard your data. This means that all data is stored in a way that prevents unauthorized access, and the use of blockchain ensures that every action involving your data is logged and cannot be changed, providing a secure and transparent environment."

User Question: "What are the key takeaways from the whitepaper?"
Prism Response: "The key points of the Prismo Technology whitepaper are: first, the introduction of the Prismo Accountability Layer, which ensures secure and transparent transactions through blockchain; second, the use of advanced security measures like encryption to protect data; and finally, how this technology aims to build a new era of trust in digital interactions."
"""

In [6]:
struct = [{"role": "system", "content": System_Prompt}]

In [7]:
dataframed = pd.read_csv('https://raw.githubusercontent.com/PrismoProtocolDevelopment/Prism_Prismo_Technology/refs/heads/main/Dataset/Prismo_Knowledgebase.csv')

In [8]:
dataframed['combined'] = dataframed.apply(lambda row : ' '.join(row.values.astype(str)), axis = 1)
documents = dataframed['combined'].tolist()
embeddings = [get_embedding(doc, engine = "text-embedding-ada-002") for doc in documents]
embedding_dim = len(embeddings[0])
embeddings_np = np.array(embeddings).astype('float32')
index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings_np)

In [9]:
user_message = "What is the Prismo Accountability Layer?"

In [10]:
query_embedding = get_embedding(user_message, engine='text-embedding-3-small')
query_embedding_np = np.array([query_embedding]).astype('float32')
_, indices = index.search(query_embedding_np, 5)
retrieved_docs = [documents[i] for i in indices[0]]
context = ' '.join(retrieved_docs)
structured_prompt = f"Context:\n{context}\n\nQuery:\n{user_message}\n\nResponse:"

In [11]:
chat =  openai.ChatCompletion.create(model = "gpt-4o-mini", messages = struct + [{"role": "user", "content" : structured_prompt}], temperature=0.5, max_tokens=1500, top_p=1, frequency_penalty=0, presence_penalty=0)
struct.append({"role": "user", "content": user_message})
response = chat.choices[0].message.content
struct.append({"role": "assistant", "content": response})

In [12]:
print(response)

The Prismo Accountability Layer is a key feature of Prismo Technology that ensures every transaction within its ecosystem is secure, transparent, and traceable. It utilizes blockchain technology to create an immutable record of all interactions, meaning that once something is recorded, it cannot be changed or tampered with. This layer enhances trust among users, as it allows anyone to verify the actions taken with their data and ensures that all transactions are conducted fairly and openly. In essence, the Prismo Accountability Layer is designed to foster a secure environment for digital interactions, reinforcing Prismo's commitment to transparency and integrity.


In [13]:
encoded_text = [ord(char) for char in response]

In [14]:
print(f"Encoded Text: {encoded_text}")

Encoded Text: [84, 104, 101, 32, 80, 114, 105, 115, 109, 111, 32, 65, 99, 99, 111, 117, 110, 116, 97, 98, 105, 108, 105, 116, 121, 32, 76, 97, 121, 101, 114, 32, 105, 115, 32, 97, 32, 107, 101, 121, 32, 102, 101, 97, 116, 117, 114, 101, 32, 111, 102, 32, 80, 114, 105, 115, 109, 111, 32, 84, 101, 99, 104, 110, 111, 108, 111, 103, 121, 32, 116, 104, 97, 116, 32, 101, 110, 115, 117, 114, 101, 115, 32, 101, 118, 101, 114, 121, 32, 116, 114, 97, 110, 115, 97, 99, 116, 105, 111, 110, 32, 119, 105, 116, 104, 105, 110, 32, 105, 116, 115, 32, 101, 99, 111, 115, 121, 115, 116, 101, 109, 32, 105, 115, 32, 115, 101, 99, 117, 114, 101, 44, 32, 116, 114, 97, 110, 115, 112, 97, 114, 101, 110, 116, 44, 32, 97, 110, 100, 32, 116, 114, 97, 99, 101, 97, 98, 108, 101, 46, 32, 73, 116, 32, 117, 116, 105, 108, 105, 122, 101, 115, 32, 98, 108, 111, 99, 107, 99, 104, 97, 105, 110, 32, 116, 101, 99, 104, 110, 111, 108, 111, 103, 121, 32, 116, 111, 32, 99, 114, 101, 97, 116, 101, 32, 97, 110, 32, 105, 109, 109,

In [15]:
HE = Pyfhel()
HE.contextGen(scheme='BFV', n=2**14, t_bits=20)
HE.keyGen()

cipher_text = [HE.encryptInt(np.array([num], dtype=np.int64)) for num in encoded_text]
print(f"Cipher Text: {[str(ct) for ct in cipher_text]}")

Cipher Text: ['<Pyfhel Ciphertext at 0x7bc6704328e0, scheme=bfv, size=2/2, noiseBudget=361>', '<Pyfhel Ciphertext at 0x7bc66fcfcfe0, scheme=bfv, size=2/2, noiseBudget=361>', '<Pyfhel Ciphertext at 0x7bc66fcfe2a0, scheme=bfv, size=2/2, noiseBudget=361>', '<Pyfhel Ciphertext at 0x7bc66fcfe520, scheme=bfv, size=2/2, noiseBudget=361>', '<Pyfhel Ciphertext at 0x7bc66fcfd3f0, scheme=bfv, size=2/2, noiseBudget=361>', '<Pyfhel Ciphertext at 0x7bc66fcfd940, scheme=bfv, size=2/2, noiseBudget=361>', '<Pyfhel Ciphertext at 0x7bc6ae397ba0, scheme=bfv, size=2/2, noiseBudget=361>', '<Pyfhel Ciphertext at 0x7bc66fcfce00, scheme=bfv, size=2/2, noiseBudget=361>', '<Pyfhel Ciphertext at 0x7bc6704309f0, scheme=bfv, size=2/2, noiseBudget=361>', '<Pyfhel Ciphertext at 0x7bc66fcfc090, scheme=bfv, size=2/2, noiseBudget=361>', '<Pyfhel Ciphertext at 0x7bc66fcfcc70, scheme=bfv, size=2/2, noiseBudget=361>', '<Pyfhel Ciphertext at 0x7bc66fcffc40, scheme=bfv, size=2/2, noiseBudget=361>', '<Pyfhel Ciphertext at 0x7

In [18]:
print(str(cipher_text[0]))

<Pyfhel Ciphertext at 0x7bc6704328e0, scheme=bfv, size=2/2, noiseBudget=361>


In [16]:
decrypted_text = [HE.decryptInt(ct)[0] for ct in cipher_text]
decoded_text = ''.join([chr(num) for num in decrypted_text])

print(f"Original Text: {response}")
print("")
print(f"Decoded Text after encryption/decryption: {decoded_text}")

Original Text: The Prismo Accountability Layer is a key feature of Prismo Technology that ensures every transaction within its ecosystem is secure, transparent, and traceable. It utilizes blockchain technology to create an immutable record of all interactions, meaning that once something is recorded, it cannot be changed or tampered with. This layer enhances trust among users, as it allows anyone to verify the actions taken with their data and ensures that all transactions are conducted fairly and openly. In essence, the Prismo Accountability Layer is designed to foster a secure environment for digital interactions, reinforcing Prismo's commitment to transparency and integrity.

Decoded Text after encryption/decryption: The Prismo Accountability Layer is a key feature of Prismo Technology that ensures every transaction within its ecosystem is secure, transparent, and traceable. It utilizes blockchain technology to create an immutable record of all interactions, meaning that once someth